In [1]:
import sys
sys.path.append("..")

from typing import List, Dict, Callable, Optional
from datetime import datetime, timedelta
from IPython.display import display

import numpy as np
import pandas as pd
import MetaTrader5 as mt5

from sesto.constants import CURRENCIES
import sesto.metatrader.data as mtd
from sesto.plot import plot_tradingview, plot_plotly
from sesto.indicators import SMA, EMA, RSI, ROC, MACD
from sesto.backtester import Backtester, Trade

MetaTrader 5 initialized successfully.


In [2]:
# GENERATE ALL PAIR COMBINATIONS
from itertools import permutations

print(f"Number of currencies: {len(CURRENCIES)}")
# Generate all unique pairs of currency symbols
pairs = list(permutations(CURRENCIES, 2))
print(f"Number of pairs: {len(pairs)}")

Number of currencies: 9
Number of pairs: 72


In [3]:
# concatenate pairs into a single list
PAIRS = [pair[0] + pair[1] for pair in pairs]

In [4]:
TIMEFRAME = mt5.TIMEFRAME_H1
DATA_FROM_DATE = datetime(2024, 4, 5)
DATA_TO_DATE = datetime(2024, 9, 20)
MA_PERIODS = [7, 14, 21]

CSM_PERIOD = 21

CSM_TOP_THRESHOLD = 65
CSM_LOW_THRESHOLD = 35
CSM_TOP_REVERSAL_THRESHOLD = 85
CSM_LOW_REVERSAL_THRESHOLD = 15

HISTOGRAM_THRESHOLD = 0.4


In [5]:
BACKTEST_FROM_DATE = datetime(2024, 1, 1)
BACKTEST_TO_DATE = datetime(2024, 6, 1)

INITIAL_CAPITAL = 10000

CAPITAL_PER_TRADE = 200  # $200 per trade
TRANSACTION_COST = 0.0001
SLIPPAGE = 0.0001
TRADE_SL_MULTIPLIER = 0.0025
RR_MULTIPLIER = 2
LEVERAGE = 500

In [6]:
mtd.fill_data_range(PAIRS, TIMEFRAME, DATA_FROM_DATE, DATA_TO_DATE)

Failed to fetch data for symbol: USDEUR
No data fetched for pair: USDEUR
Failed to fetch data for symbol: USDGBP
No data fetched for pair: USDGBP
Fetched data for pair: USDJPY
Fetched data for pair: USDCHF
Fetched data for pair: USDCAD
Failed to fetch data for symbol: USDAUD
No data fetched for pair: USDAUD
Fetched data for pair: USDCHF
Failed to fetch data for symbol: USDNZD
No data fetched for pair: USDNZD
Fetched data for pair: EURUSD
Fetched data for pair: EURGBP
Fetched data for pair: EURJPY
Fetched data for pair: EURCHF
Fetched data for pair: EURCAD
Fetched data for pair: EURAUD
Fetched data for pair: EURCHF
Fetched data for pair: EURNZD
Fetched data for pair: GBPUSD
Failed to fetch data for symbol: GBPEUR
No data fetched for pair: GBPEUR
Fetched data for pair: GBPJPY
Fetched data for pair: GBPCHF
Fetched data for pair: GBPCAD
Fetched data for pair: GBPAUD
Fetched data for pair: GBPCHF
Fetched data for pair: GBPNZD
Failed to fetch data for symbol: JPYUSD
No data fetched for pair:

In [7]:
pairs_to_delete = [pair for pair, data in mtd.data.items() if data is None or len(data) == 0]
for pair in pairs_to_delete:
    del mtd.data[pair]

# Get the length of valid pair keys in mtd.data that have data
valid_pairs = sum(1 for pair, data in mtd.data.items() if data is not None and len(data) > 0)
print(f"Number of valid pairs with data: {valid_pairs}")

Number of valid pairs with data: 28


In [8]:
for symbol, df in mtd.data.items():
    for period in MA_PERIODS:
        SMA(df, period)
        EMA(df, period)
        ROC(df, period)
        RSI(df, period)
    
    df['psm'] = df[f'rsi-{CSM_PERIOD}'] + df[f'roc-{CSM_PERIOD}'] / 2


In [9]:
csm_data = {}

for currency in CURRENCIES:
    # Initialize a list to hold DataFrames of PSM values for the current currency
    psm_dfs = []
    
    # Iterate over all pair data
    for pair, df in mtd.data.items():
        if df is None or df.empty:
            continue
        
        base_currency = pair[:3]
        quote_currency = pair[3:]
        
        # Determine if the current currency is the base or quote in the pair
        if currency == base_currency:
            # Use PSM as is for base currency
            psm_series = df[['time', 'psm']].copy()
        elif currency == quote_currency:
            # Invert PSM for quote currency
            psm_series = df[['time', 'psm']].copy()
            psm_series['psm'] = -psm_series['psm']
        else:
            # Current currency not involved in this pair
            continue
        
        # Rename the 'psm' column to the pair name
        psm_series.rename(columns={'psm': pair}, inplace=True)
        
        # Append to the list of PSM DataFrames
        psm_dfs.append(psm_series)
    
    if not psm_dfs:
        print(f"No pairs found for currency: {currency}")
        continue
    
    # Merge all PSM DataFrames on 'time' using outer join to preserve all timestamps
    csm_df = psm_dfs[0]
    for psm_df in psm_dfs[1:]:
        csm_df = pd.merge(csm_df, psm_df, on='time', how='outer')
    
    # Sort by time to maintain chronological order
    csm_df.sort_values('time', inplace=True)
    
    # Reset index after sorting
    csm_df.reset_index(drop=True, inplace=True)
    
    # Handle missing PSM values by excluding them from the mean calculation
    # (NaNs are automatically ignored by pandas' mean function with skipna=True)
    
    # Calculate the mean PSM value across all relevant pairs for each candle
    csm_df['csm'] = csm_df.drop('time', axis=1).mean(axis=1)

    # Smooth CSM
    csm_df['csm'] = csm_df['csm'].rolling(CSM_PERIOD).mean()

    min_csm = csm_df['csm'].min()
    max_csm = csm_df['csm'].max()
    csm_df['csm'] = 100 * (csm_df['csm'] - min_csm) / (max_csm - min_csm)
        
    # Store the CSM DataFrame in the dictionary
    csm_data[currency] = csm_df

In [10]:
for currency, df in csm_data.items():
    MACD(df = df, column = 'csm', short_period=12, long_period=26, signal_period=9)


In [11]:
currencies_to_plot = ['GBP', 'JPY']  # You can modify this list as needed

# Create a new DataFrame to hold CSM data for all specified currencies
combined_csm_df = pd.DataFrame()

for currency in currencies_to_plot:
    if currency in csm_data:
        # Add the CSM data for this currency to the combined DataFrame
        combined_csm_df[currency] = csm_data[currency]['csm']
        
        # If we haven't set the 'time' column yet, use this currency's time data
        if 'time' not in combined_csm_df.columns:
            combined_csm_df['time'] = csm_data[currency]['time']

# Sort the combined DataFrame by time
combined_csm_df.sort_values('time', inplace=True)

# Plot the combined CSM data
plot_plotly(combined_csm_df, 'Combined CSM', currencies_to_plot)

In [12]:
def get_csm(symbol: str, row: pd.Series):
    base_currency = symbol[:3]
    quote_currency = symbol[3:]

    base_csm_df = csm_data[base_currency]
    quote_csm_df = csm_data[quote_currency]

    base_csm_row = base_csm_df.loc[base_csm_df['time'] == row['time']].iloc[0]
    quote_csm_row = quote_csm_df.loc[quote_csm_df['time'] == row['time']].iloc[0]

    base_csm = base_csm_row['csm']
    quote_csm = quote_csm_row['csm']

    return base_csm_df, quote_csm_df, base_csm_row, quote_csm_row, base_csm, quote_csm

def get_macd(symbol: str, row: pd.Series):
    base_currency = symbol[:3]
    quote_currency = symbol[3:]

    base_csm_df = csm_data[base_currency]
    quote_csm_df = csm_data[quote_currency]

    base_csm_row = base_csm_df.loc[base_csm_df['time'] == row['time']].iloc[0]
    quote_csm_row = quote_csm_df.loc[quote_csm_df['time'] == row['time']].iloc[0]

    base_macd = base_csm_row['macd']
    quote_macd = quote_csm_row['macd']

    base_macd_signal = base_csm_row['macd-signal']
    quote_macd_signal = quote_csm_row['macd-signal']

    base_macd_histogram = base_csm_row['macd-histogram']
    quote_macd_histogram = quote_csm_row['macd-histogram']

    base_macd_bullish = base_macd > base_macd_signal and base_macd_histogram > 0
    quote_macd_bullish = quote_macd > quote_macd_signal and quote_macd_histogram > 0

    base_macd_bearish = base_macd < base_macd_signal and base_macd_histogram < 0
    quote_macd_bearish = quote_macd < quote_macd_signal and quote_macd_histogram < 0

    return base_macd, quote_macd, base_macd_signal, quote_macd_signal, base_macd_histogram, quote_macd_histogram, base_macd_bullish, quote_macd_bullish, base_macd_bearish, quote_macd_bearish

class MyStrategy(Backtester):
    def entry_condition(self, symbol: str, time: datetime, row: pd.Series, open_trades: List[Trade], all_trades: List[Trade]) -> Optional[Dict]:
        base_currency = symbol[:3]
        quote_currency = symbol[3:]

        base_csm_df = csm_data[base_currency]
        quote_csm_df = csm_data[quote_currency]

        if row['time'] not in base_csm_df['time'].values or row['time'] not in quote_csm_df['time'].values:
            return None

        base_csm_row = base_csm_df.loc[base_csm_df['time'] == row['time']].iloc[0]
        quote_csm_row = quote_csm_df.loc[quote_csm_df['time'] == row['time']].iloc[0]

        base_csm = base_csm_row['csm']
        quote_csm = quote_csm_row['csm']

        base_macd, quote_macd, base_macd_signal, quote_macd_signal, base_macd_histogram, quote_macd_histogram, base_macd_bullish, quote_macd_bullish, base_macd_bearish, quote_macd_bearish = get_macd(symbol, row)
        
        symbol_open_trades = [trade for trade in open_trades if trade.symbol == symbol]

        if len(symbol_open_trades) == 0:
            # Long Position Trade in the direction of the trend
            if base_csm > CSM_TOP_THRESHOLD and quote_csm < CSM_LOW_THRESHOLD and base_macd_bullish and quote_macd_bullish:
                return {
                    'entry_price': row['close'],
                    'position_type': 'long',
                    'capital_allocation': CAPITAL_PER_TRADE,
                    'tp': row['close'] * (1 + (TRADE_SL_MULTIPLIER * RR_MULTIPLIER)),
                    'sl': row['close'] * (1 - TRADE_SL_MULTIPLIER)
                }

            # Short Position Trade in the direction of the trend
            if base_csm < CSM_LOW_THRESHOLD and quote_csm > CSM_TOP_THRESHOLD and base_macd_bearish and quote_macd_bearish:
                return {

                    'entry_price': row['close'],
                    'position_type': 'short',
                    'capital_allocation': CAPITAL_PER_TRADE,
                    'tp': row['close'] * (1 - (TRADE_SL_MULTIPLIER * RR_MULTIPLIER)),
                    'sl': row['close'] * (1 + TRADE_SL_MULTIPLIER)
                }
            
            # Long Position Trade in the opposite direction of the trend - Reversal 
            if base_csm < CSM_LOW_REVERSAL_THRESHOLD and base_macd_bullish and quote_csm > CSM_TOP_REVERSAL_THRESHOLD and quote_macd_bearish:
                return {
                    'entry_price': row['close'],
                    'position_type': 'long',
                    'capital_allocation': CAPITAL_PER_TRADE,
                    'tp': row['close'] * (1 + (TRADE_SL_MULTIPLIER * RR_MULTIPLIER)),
                    'sl': row['close'] * (1 - TRADE_SL_MULTIPLIER)
                }
            
            # Short Position Trade in the opposite direction of the trend - Reversal 
            if base_csm > CSM_TOP_REVERSAL_THRESHOLD and base_macd_bearish and quote_csm < CSM_LOW_REVERSAL_THRESHOLD and quote_macd_bullish:
                return {
                    'entry_price': row['close'],
                    'position_type': 'short',
                    'capital_allocation': CAPITAL_PER_TRADE,
                    'tp': row['close'] * (1 - (TRADE_SL_MULTIPLIER * RR_MULTIPLIER)),
                    'sl': row['close'] * (1 + TRADE_SL_MULTIPLIER)
                }


        return None

        
    def exit_condition(self, trade: Trade, time: datetime, row: pd.Series, open_trades: List[Trade], closed_trades: List[Trade]) -> bool:

        return False

    def trailing_stop(self, trade: Trade, time: datetime, row: pd.Series, open_trades: List[Trade], closed_trades: List[Trade]):
        # Implement your trailing stop logic here
        pass

In [13]:
backtest = MyStrategy(
    mtd.data, 
    initial_capital=INITIAL_CAPITAL, 
    transaction_cost=TRANSACTION_COST, 
    slippage=SLIPPAGE,
    leverage=LEVERAGE
)
backtest.run()

Opened long trade for USDJPY at 154.507. Available capital: 9998.58
Potential profit: $500.00 (35115.23%)
Potential loss: $250.00 (17557.61%)
TP price diff: 0.50%, SL price diff: 0.25%
Closed long trade for USDJPY at 154.1207325 with PNL -1.75. Available capital: 9998.25
Opened short trade for USDJPY at 156.677. Available capital: 9996.85
Potential profit: $500.00 (35608.41%)
Potential loss: $250.00 (17804.20%)
TP price diff: 0.50%, SL price diff: 0.25%
Closed short trade for USDJPY at 157.0686925 with PNL -1.72. Available capital: 9996.53
Opened short trade for USDJPY at 150.17. Available capital: 9995.06
Potential profit: $500.00 (34129.55%)
Potential loss: $250.00 (17064.77%)
TP price diff: 0.50%, SL price diff: 0.25%
Closed short trade for USDJPY at 149.41914999999997 with PNL 3.20. Available capital: 9999.73
Opened short trade for USDJPY at 149.173. Available capital: 9998.25
Potential profit: $500.00 (33902.95%)
Potential loss: $250.00 (16951.48%)
TP price diff: 0.50%, SL price d

In [14]:
trade_log = pd.DataFrame(backtest.trade_log)
trade_log

,symbol,entry_time,entry_price,position_type,position_size_usd,tp,sl,used_capital,potential_profit_usd,potential_profit_percent,...,potential_loss_percent,tp_price_diff_percent,sl_price_diff_percent,close_time,close_price,pnl,max_drawdown,max_profit,closing_reason,unrealized_pnl
0,USDJPY,2024-04-16 09:00:00,154.50700,long,647.219867,155.279535,154.120733,1.423884,500.0,35115.227273,...,17557.613636,0.5,0.25,2024-04-19 05:00:00,154.120733,-1.747494,0,0,sl,0
1,USDJPY,2024-05-31 04:00:00,156.67700,short,638.255775,155.893615,157.068692,1.404163,500.0,35608.409091,...,17804.204545,0.5,0.25,2024-05-31 10:00:00,157.068692,-1.723291,0,0,sl,0
2,USDJPY,2024-08-01 01:00:00,150.17000,short,665.911966,149.419150,150.545425,1.465006,500.0,34129.545455,...,17064.772727,0.5,0.25,2024-08-01 04:00:00,149.419150,3.196377,0,0,tp,0
3,USDJPY,2024-08-01 04:00:00,149.17300,short,670.362599,148.427135,149.545932,1.474798,500.0,33902.954545,...,16951.477273,0.5,0.25,2024-08-01 05:00:00,149.545932,-1.809979,0,0,sl,0
4,USDJPY,2024-08-01 05:00:00,149.76700,short,667.703833,149.018165,150.141417,1.468948,500.0,34037.954545,...,17018.977273,0.5,0.25,2024-08-01 14:00:00,150.141417,-1.802800,0,0,sl,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,NZDCAD,2024-07-08 14:00:00,0.83636,long,119565.737242,0.840542,0.834269,263.044622,500.0,190.081818,...,95.040909,0.5,0.25,2024-07-09 11:00:00,0.834269,-322.827491,0,0,sl,0
324,NZDCAD,2024-07-19 16:00:00,0.82631,short,121019.956191,0.822178,0.828376,266.243904,500.0,187.797727,...,93.898864,0.5,0.25,2024-07-22 18:00:00,0.822178,580.895790,0,0,tp,0
325,NZDCAD,2024-08-14 01:00:00,0.83257,long,120110.020779,0.836733,0.830489,264.242046,500.0,189.220455,...,94.610227,0.5,0.25,2024-08-14 05:00:00,0.830489,-324.297056,0,0,sl,0
326,NZDCAD,2024-08-20 08:00:00,0.83440,long,119846.596357,0.838572,0.832314,263.662512,500.0,189.636364,...,94.818182,0.5,0.25,2024-08-21 03:00:00,0.838572,575.263663,0,0,tp,0


In [15]:
performance_report = backtest.generate_report()
performance_report

,Metric,Value
0,Initial Capital,$10000.00
1,Final Capital,$16798.85
2,Total Profit,$6798.85
3,Return (%),67.99%
4,Annualized Return (%),233.99%
5,Volatility (Ann.),155.79
6,Sharpe Ratio,1.49
7,Sortino Ratio,3.37
8,Calmar Ratio,8.92
9,Max. Drawdown ($),$-2622.15
